# Make Figure 3A: EGFR mutation status

This notebooks makes Figure 3A. It shows the mutation type abundance for EGFR across the different cancers. 

In [1]:
import cptac
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plot_utils as p


/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
cptac warning: Your version of cptac (0.8.7) is out-of-date. Latest is 0.8.8. Please run 'pip install --upgrade cptac' to update it. (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


In [2]:
import warnings
warnings.filterwarnings('ignore')
#cptac version
cptac.version()   

'0.8.7'

# Step 1 Create data frames with Mutation Types 

For each cancer type we need to create a df that has the mutation type for each sample. 


First, load in cptac datasets

In [3]:
en = cptac.Endometrial()
hn = cptac.Hnscc()
l = cptac.Luad()
o = cptac.Ovarian()
c = cptac.Ccrcc()
col = cptac.Colon()
g = cptac.Gbm()
b = cptac.Brca()
ls = cptac.Lscc()

Second, call get_geneotype_all_vars for EGFR for each cancer type. This returns a df with mutation, location, and mutation_status. For individuals with multiple mutations, we report a single mutation type based on the following priority: deletion, truncation, missense, amplification, inframe, silent, and wildtype.

In [58]:
def figure1_plot_mutations(dflist = None, names_of_df=None, title=None, save_to_path=None):
    number_of_df = len(dflist)

    allLabels = []
    for df in dflist:
        #get the labels for each and make a combined label that they'll all use
        mutation = df["Mutation"]
        labels = list(set(mutation))

        allLabels.append(labels)

    flat_list = [item for sublist in allLabels for item in sublist]
    all_labels = list(set(flat_list))
    all_labels.sort()
    allLabels = all_labels

#     For each df, add na to their labels if it doesn't exist in all_labels
    labels_for_each_df = []
    frequencies_for_each_df = []
    for df in dflist:
        mutation = df["Mutation"].tolist()
        mutationlist = list(set(mutation))
        mutationlist.sort()
        ordered_mut_list = []
        match = True
        mutPosition = 0

        for position in range(len(all_labels)):
            try:

                if mutationlist[mutPosition] == all_labels[position]:
                    ordered_mut_list.append(mutationlist[mutPosition])
                    mutPosition += 1

                else:
                    ordered_mut_list.append("na")

            except IndexError:
                ordered_mut_list.append("na")


        labels_for_each_df.append(ordered_mut_list)

        #get the freq of each mutation type
        freq = []
        for mutation_type in ordered_mut_list:
            freq.append(mutation.count(mutation_type))

        PercentFreq = [x*100 / sum(freq) for x in freq]
        frequencies_for_each_df.append(PercentFreq)


    #Now plot it using arrays
    width = .11
    x = np.arange(len(allLabels))
    a4_dims = (150, 90) #dimensions for bigger plot
    fig, ax = plt.subplots(figsize=a4_dims)
    for position in range(0, number_of_df):
        r = ax.bar(x+(width*position), frequencies_for_each_df[position], width,label=names_of_df[position], alpha=.5, linewidth=0)



    ax.set_ylabel('Percent Sample', size = 300)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(allLabels)
    ax.legend()
    ax.set_xticklabels(allLabels, size = 300)
    ax.set_ylim(0,100)
    ax.legend(prop={'size': 270})
    ax.yaxis.set_tick_params(labelsize=300)


    plt.setp(ax.get_xticklabels(),rotation= 45)
    plt.tight_layout()
    if save_to_path == None:
        plt.savefig("step_1.png",dpi = 300)
    else:
        plt.savefig(save_to_path)

    plt.show()


In [5]:
endo = en.get_genotype_all_vars("EGFR")
gbm = g.get_genotype_all_vars("EGFR")
hnscc = hn.get_genotype_all_vars("EGFR")
ovar = o.get_genotype_all_vars("EGFR")
cc = c.get_genotype_all_vars("EGFR")
colon = col.get_genotype_all_vars("EGFR")
ld = l.get_genotype_all_vars("EGFR")
brca = b.get_genotype_all_vars("EGFR")
lscc = ls.get_genotype_all_vars("EGFR")

In [6]:
#example 
gbm.head()

Name                 Mutation       Location    Mutation_Status
Patient_ID                                                     
C3L-00104       Amplification  Amplification    Single_mutation
C3L-00365   Missense_Mutation        p.G665D  Multiple_mutation
C3L-00674       Amplification  Amplification    Single_mutation
C3L-00677       Amplification  Amplification    Single_mutation
C3L-01040       Amplification  Amplification    Single_mutation

# Step 2: Match Mutation Labels

Change labels to to match desired final figure. 

In [7]:
# Simplify mutation names
dfs = [endo, gbm, hnscc, ovar, cc, colon, ld, brca, lscc, cc]
for df in dfs:
    df['Mutation'].where(df['Mutation'] != 'Missense_Mutation', 'Missense', inplace = True) # replaces when false
    df['Mutation'].where(df['Mutation'] != 'Wildtype_Tumor', 'Wildtype', inplace = True)
    df['Mutation'].where(df['Mutation'] != 'In_Frame_Del', 'Indel', inplace = True)
    df['Mutation'].where(df['Mutation'] != 'Nonsense_Mutation', 'Truncation', inplace = True)
    
# change colon labels to match other cancers. Nonsynonymous in this case is missense and nonframeshift insertion is Indel.
colon["Mutation"] = colon['Mutation'].replace(['nonsynonymous SNV'], 'Missense')
colon["Mutation"] = colon['Mutation'].replace(['nonframeshift insertion'], 'Indel')

In [8]:

#Drop Silent mutation type
indexNames = hnscc[ hnscc['Mutation'] == "Silent" ].index
hnscc.drop(indexNames , inplace=True)
indexNames = ld[ ld['Mutation'] == "Silent" ].index
ld.drop(indexNames , inplace=True)
indexNames = brca[ brca['Mutation'] == "Silent" ].index
brca.drop(indexNames , inplace=True)


In [9]:
endo

Name          Mutation     Location    Mutation_Status
Patient_ID                                            
C3L-00006     Wildtype  No_mutation     Wildtype_Tumor
C3L-00008     Wildtype  No_mutation     Wildtype_Tumor
C3L-00032     Wildtype  No_mutation     Wildtype_Tumor
C3L-00090     Wildtype  No_mutation     Wildtype_Tumor
C3L-00098     Wildtype  No_mutation     Wildtype_Tumor
...                ...          ...                ...
C3N-01520   Truncation     p.Q1067*  Multiple_mutation
C3N-01521     Wildtype  No_mutation     Wildtype_Tumor
C3N-01537     Wildtype  No_mutation     Wildtype_Tumor
C3N-01802     Wildtype  No_mutation     Wildtype_Tumor
C3N-01825     Wildtype  No_mutation     Wildtype_Tumor

[95 rows x 3 columns]

# Step 3: Create figure 

Put dfs into a list. Create list of names for figure legend. 
Call plot_mutations function.

In [59]:
dfs = [endo, gbm, hnscc, ovar, cc, colon, ld, brca, lscc]
names = ['Endometrial', 'Glioblastoma', 'Head and Neck', 'Ovarian', 'Renal', 'Colon', 'Lung Adenocarcinoma', 'Breast',"Lung Squamous"]
figure1_plot_mutations(dfs, names, save_to_path = "png_files/Figure3A.png")

# Calculate Percent of samples with Amplification mutations

The manuscript mentions the exact percentage of EGFR amplification mutations. Here is where we derive those percentages. 

In [9]:
#Calculate exact percent
i = 0
for cancer in dfs:

    vc = cancer.Mutation.value_counts()
    total = len(cancer)
    cnv_amp = vc.Amplification
    print(names[i])
    print('total_samples:',total,'\n')
    print(cnv_amp,'/',total,'=', cnv_amp/total,'\n')
    i += 1
   

Endometrial
total_samples: 95 

8 / 95 = 0.08421052631578947 

Glioblastoma
total_samples: 99 

66 / 99 = 0.6666666666666666 

Head and Neck
total_samples: 109 

31 / 109 = 0.28440366972477066 

Ovarian
total_samples: 102 

23 / 102 = 0.22549019607843138 

Renal
total_samples: 110 

28 / 110 = 0.2545454545454545 

Colon
total_samples: 106 

65 / 106 = 0.6132075471698113 

Lung Adenocarcinoma
total_samples: 108 

26 / 108 = 0.24074074074074073 

Breast
total_samples: 121 

31 / 121 = 0.256198347107438 

Lung Squamous
total_samples: 108 

37 / 108 = 0.3425925925925926 



In [10]:
#Calculate exact percent
i = 0
for cancer in dfs:

    vc = cancer.Mutation.value_counts()
    total = len(cancer)
    cnv_amp = vc.Amplification
    print(names[i])
    print('total_samples:',total,'\n')
    print(cnv_amp,'/',total,'=', cnv_amp/total,'\n')
    i += 1
   

Endometrial
total_samples: 95 

8 / 95 = 0.08421052631578947 

Glioblastoma
total_samples: 99 

66 / 99 = 0.6666666666666666 

Head and Neck
total_samples: 109 

31 / 109 = 0.28440366972477066 

Ovarian
total_samples: 102 

23 / 102 = 0.22549019607843138 

Renal
total_samples: 110 

28 / 110 = 0.2545454545454545 

Colon
total_samples: 106 

65 / 106 = 0.6132075471698113 

Lung Adenocarcinoma
total_samples: 108 

26 / 108 = 0.24074074074074073 

Breast
total_samples: 121 

31 / 121 = 0.256198347107438 

Lung Squamous
total_samples: 108 

37 / 108 = 0.3425925925925926 

